In [1]:
!unzip '*.zip'

Archive:  title_index.zip
   creating: title_index/
  inflating: title_index/title.pkl   
  inflating: title_index/title_index_31_000.bin  
  inflating: title_index/title_index_9_000.bin  
  inflating: title_index/title_index_40_000.bin  
  inflating: title_index/title_index_80_000.bin  
  inflating: title_index/title_index_55_000.bin  
  inflating: title_index/title_index_115_000.bin  
  inflating: title_index/title_index_98_000.bin  
  inflating: title_index/title_index_59_000.bin  
  inflating: title_index/title_index_45_000.bin  

Archive:  text_index.zip
   creating: text_index/
  inflating: text_index/text_index_59_000.bin  
  inflating: text_index/text_index_108_000.bin  
  inflating: text_index/text_index_50_000.bin  
  inflating: text_index/text_index_42_000.bin  
  inflating: text_index/text_index_101_000.bin  
  inflating: text_index/text_index_76_000.bin  
  inflating: text_index/text_index_47_000.bin  
  inflating: text_index/text_index_1_000.bin  
  inflating: text_index/

In [2]:
import sys
# from collections import Counter, OrderedDict
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab_edited import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# These will already be installed in the testing environment so disregard the
# amount of time (~1 minute) it takes to install.
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 124565 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u432-ga~us1-0ubuntu2~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u432-ga~us1-0ub

In [4]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [5]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [6]:
spark

In [7]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 100

def handler(signum, frame):
  raise Exception("Authentication timeout!")

try:
  signal.signal(signal.SIGALRM, handler)
  signal.alarm(AUTH_TIMEOUT)
  auth.authenticate_user()
  signal.alarm(0)
except:
  pass

In [8]:
import os
from pathlib import Path
from google.colab import auth
from google.cloud import storage

# Alternative authentication method
!gcloud auth login

# Set your project and bucket info
project_id = 'hw2-project-444515'
data_bucket_name = '314616475'

# Configure project
!gcloud config set project {project_id}

# Create directory and copy file directly
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass
except:
    !mkdir -p wikidumps
    !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/"


You are running on a Google Compute Engine virtual machine.
It is recommended that you use service accounts for authentication.

You can run:

  $ gcloud config set account `ACCOUNT`

to switch accounts if necessary.

Your credentials may be visible to others with access to this
virtual machine. Are you sure you want to authenticate with
your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.

In [9]:
from pathlib import Path
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [10]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd
doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd
doc_anchor_pairs = parquetFile.limit(1000).select("anchor_text", "id").rdd

In [14]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)

def tokenize(text):
  english_stopwords = frozenset(stopwords.words('english'))
  corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
  RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

  all_stopwords = english_stopwords.union(corpus_stopwords)
  stemmer = PorterStemmer()
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  stemmed = [stemmer.stem(token) for token in tokens if token not in all_stopwords]
  return stemmed

def stemmed_word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = tokenize(text)
  token_counts = Counter(tokens)  # Calculate term frequencies efficiently
  res_list = [
        (token, (id, count))
        for token, count in token_counts.items()
        if token not in all_stopwords
    ]
  return res_list


def len_tokens(text, id):
  english_stopwords = frozenset(stopwords.words('english'))
  corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
  RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
  all_stopwords = english_stopwords.union(corpus_stopwords)

  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  return (id, len(tokens))

In [15]:
text_doc_lengths_dict = {}
title_doc_lengths_dict = {}


# Apply the function to your RDD
text_len_pairs = doc_text_pairs.map(lambda x: len_tokens(x[0], x[1]))
text_doc_lengths_dict = text_len_pairs.collectAsMap()

title_len_pairs = doc_title_pairs.map(lambda x: len_tokens(x[0], x[1]))
title_doc_lengths_dict = title_len_pairs.collectAsMap()

In [16]:
import math
from itertools import chain
import time
import builtins


# When preprocessing the data have a dictionary of document length for each document saved in a variable called `DL`.
class BM25_from_index:
    """
    Best Match 25.
    ----------
    k1 : float, default 1.5

    b : float, default 0.75

    index: inverted index
    """

    def __init__(self,index,DL,k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1
        self.index = index
        self.DL = DL
        self.N = len(DL)
        # self.AVGDL = sum(DL.values())/self.N
        # self.AVGDL = sum(list(DL.values())) / self.N
        self.AVGDL = builtins.sum(DL.values()) / self.N
        self.words, self.pls = zip(*self.index.posting_lists_iter())

    def calc_idf(self,list_of_tokens):
        """
        This function calculate the idf values according to the BM25 idf formula for each term in the query.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']

        Returns:
        -----------
        idf: dictionary of idf scores. As follows:
                                                    key: term
                                                    value: bm25 idf score
        """
        idf = {}
        for term in list_of_tokens:
            if term in self.index.df.keys():
                n_ti = self.index.df[term]
                idf[term] = math.log(1 + (self.N - n_ti + 0.5) / (n_ti + 0.5))
            else:
                pass
        return idf


    def search(self, queries,N=3):
        """
        This function calculate the bm25 score for given query and document.
        We need to check only documents which are 'candidates' for a given query.
        This function return a dictionary of scores as the following:
                                                                    key: query_id
                                                                    value: a ranked list of pairs (doc_id, score) in the length of N.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.

        Returns:
        -----------
        score: float, bm25 score.
        """
        # YOUR CODE HERE
        query_scores = {} # init dict

        for query_id, query_tokens in queries.items():
            self.idf = self.calc_idf(query_tokens) # calc idf for current query

            if not self.idf:
                query_scores[query_id] = []  # no relevant documents
                continue

            # getting candidtaes
            candidate_docs = set()
            for term in query_tokens:
                if term in self.index.df:
                    posting_list = self.pls[self.words.index(term)]
                    # print(f"posting_list for term {term} is:{posting_list}")
                    candidate_docs.update([doc_id for doc_id, _ in posting_list])
                    # print(f"candidate_docs for term {term} are: {candidate_docs}")

            # calc common terms' data
            common_terms_data = {}
            for term in query_tokens:
                if term in self.index.df:
                    posting_list = self.pls[self.words.index(term)]
                    common_terms_data[term] = dict(posting_list)

            # calc scores for candidates
            doc_scores = {}
            for doc_id in candidate_docs:
                score = 0.0
                doc_len = self.DL[doc_id]

                for term in query_tokens:
                    if term in common_terms_data and doc_id in common_terms_data[term]:
                        freq = common_terms_data[term][doc_id]
                        numerator = self.idf[term] * freq * (self.k1 + 1)
                        denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.AVGDL)
                        score += (numerator / denominator)

                doc_scores[doc_id] = score

            sorted_scores = sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)
            top_n_docs = sorted_scores[:N]

            query_scores[query_id] = top_n_docs

        return query_scores

    def _score(self, query, doc_id):
        """
        This function calculate the bm25 score for given query and document.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.

        Returns:
        -----------
        score: float, bm25 score.
        """
        score = 0.0
        doc_len = DL[str(doc_id)]

        for term in query:
            if term in self.index.term_total.keys():
                term_frequencies = dict(self.pls[self.words.index(term)])
                if doc_id in term_frequencies.keys():
                    freq = term_frequencies[doc_id]
                    numerator = self.idf[term] * freq * (self.k1 + 1)
                    denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.AVGDL)
                    score += (numerator / denominator)
        return score

In [17]:
idx_text = InvertedIndex.read_index('text_index', 'text')
idx_title = InvertedIndex.read_index('title_index', 'title')
idx_anchor = InvertedIndex.read_index('anchor_index', 'anchor')

In [19]:
bm25_text = BM25_from_index(idx_text, text_doc_lengths_dict)
bm25_title = BM25_from_index(idx_title, title_doc_lengths_dict)
bm25_anchor = BM25_from_index(idx_anchor, text_doc_lengths_dict)  # this needed to be changed! wrong DL

In [20]:
pages_links = spark.read.parquet(path).limit(1000).select("id", "anchor_text").rdd

In [21]:
# Collect IDs from both RDDs
ids_pages_links = set(pages_links.map(lambda row: row.id).collect())
ids_doc_anchor_pairs = set(doc_anchor_pairs.map(lambda row: row.id).collect())

# Check if they are the same
ids_pages_links == ids_doc_anchor_pairs


True

In [22]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''
  # YOUR CODE HERE
  # raise NotImplementedError()

  # edges = pages.flatMap(lambda x: [(x[0], anchor) for anchor in x[1]])
  # edges = edges.mapValues(lambda x: (x[0],))
  edges = pages.flatMap(lambda x: [(x[0], anchor[0]) for anchor in x[1]])
  edges = edges.distinct()

  # vertices = edges.flatMap(lambda x:  [(x[0],), (x[1],)])
  # vertices = edges.flatMap(lambda x: [x[0], x[1][0]]).distinct()
  vertices = edges.flatMap(lambda x: [x[0], x[1]]).distinct()
  vertices = vertices.map(lambda x: Row(id=x))

  # vertices = vertices.map(lambda x: Row(id=x))
  # vertices = vertices.distinct()




  return edges, vertices

In [23]:
edges, vertices = generate_graph(pages_links)

In [24]:
edgesDF = edges.toDF(['src', 'dst']).repartition(4, 'src')
verticesDF = vertices.toDF(['id']).repartition(4, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=10)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv('pr', compression="gzip")
pr.show()

/usr/local/lib/python3.11/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+--------+------------------+
|      id|          pagerank|
+--------+------------------+
|  606848|1.9768910954777374|
|43432956|1.7183234055660317|
|37404646| 1.561180019985396|
|  148363| 1.531189053212204|
|     308| 1.523973241896932|
|     783|1.5205983727677175|
|   39235|1.5113438223085562|
|   30680| 1.506487061855029|
| 3434750|1.5042432651671351|
|   48518|1.5032093881505733|
| 5843419| 1.475564533072746|
|   25458|1.4651925575346503|
|     854| 1.433614708798739|
|   14532|1.4318560075868727|
|   11887| 1.422942493084949|
|   32927|1.4195131540605397|
| 8034453| 1.414325748041313|
|    1686|1.4005429901986137|
|   25507|1.3695924013144563|
|   61371|1.3682656561066355|
+--------+------------------+
only showing top 20 rows



In [25]:
def merge_results(title_scores,body_scores,title_weight=0.5,text_weight=0.5,N = 3):
    """
    This function merge and sort documents retrieved by its weighte score (e.g., title and body).

    Parameters:
    -----------
    title_scores: a dictionary build upon the title index of queries and tuples representing scores as follows:
                                                                            key: query_id
                                                                            value: list of pairs in the following format:(doc_id,score)

    body_scores: a dictionary build upon the body/text index of queries and tuples representing scores as follows:
                                                                            key: query_id
                                                                            value: list of pairs in the following format:(doc_id,score)
    title_weight: float, for weigted average utilizing title and body scores
    text_weight: float, for weigted average utilizing title and body scores
    N: Integer. How many document to retrieve. This argument is passed to topN function. By default N = 3, for the topN function.

    Returns:
    -----------
    dictionary of querires and topN pairs as follows:
                                                        key: query_id
                                                        value: list of pairs in the following format:(doc_id,score).
    """
    # YOUR CODE HERE
    merged_results = {} # init dict

    for query_id in title_scores:
        title_results = {doc_id: score for doc_id, score in title_scores.get(query_id, [])} # scores for titles
        body_results = {doc_id: score for doc_id, score in body_scores.get(query_id, [])} # scores for body

        all_doc_ids = set(title_results.keys()) | set(body_results.keys()) # Get all unique document IDs

        # calc combined scores
        combined_scores = {}
        for doc_id in all_doc_ids:
            title_score = title_results.get(doc_id, 0)
            body_score = body_results.get(doc_id, 0)
            combined_score = (title_score * title_weight) + (body_score * text_weight)
            combined_scores[doc_id] = combined_score

        sorted_scores = sorted(combined_scores.items(), key=lambda item: item[1], reverse=True)
        top_n_docs = sorted_scores[:N]

        merged_results[query_id] = top_n_docs

    return merged_results

In [49]:
queries = {
    1: "What is the role of feminism in modern society?",
    2: "Explain the causes and effects of the 9th-century revolutions.",
    3: "How did Lenin influence Soviet philosophy?",
    4: "What are the properties of genetic mutations in the human genome?",
    5: "Discuss the architectural significance of downtown skyscrapers.",
    6: "What were the primary causes of the Great Fire of London?",
    7: "Analyze the impact of postmodern art on 20th-century culture.",
    8: "What advancements in aerodynamics have improved airplane design?",
    9: "How does RNA differ from DNA in its function?",
    10: "Provide an overview of the contributions of Albert Einstein to physics.",
}

tokenized_queries = {qid: tokenize(query) for qid, query in queries.items()}

def tokenize_query(query):
    return tokenize(query)

tokens_query1 = tokenize_query(queries[1])
tokens_query2 = tokenize_query(queries[2])

bm25_queries_score_train_text = bm25_text.search({1: tokens_query1},N=10)
for score in bm25_queries_score_train_text.values():
    print(score)

[(573, 5.76128035298307), (12, 5.661777336976286), (1546, 5.653170053129046), (1494, 5.482040916127122), (1423, 5.421440686956775), (569, 5.367232607706526), (1214, 5.333890378050214), (2236, 5.263435513900069), (1135, 5.215230332177688), (2274, 5.207244135038309)]


In [58]:
def get_candidates(query, index):
  '''
    function that gets tokenized query and returns a set of candidate documents for retrieval
  '''
  words, pls = zip(*index.posting_lists_iter())
  candidate_docs = set()
  for term in query:
      if term in index.df:
          try:
              # Find the index of the term in the 'words' tuple
              term_index = words.index(term)
              # Use the index to access the posting list
              posting_list = pls[term_index]
              candidate_docs.update([doc_id for doc_id, _ in posting_list])
          except ValueError:
              # Handle the case where the term is not found in 'words'
              pass  # Or print a message, or raise an exception, depending on your needs
  return candidate_docs


print(get_candidates(tokens_query1, idx_text))

{2052, 2053, 12, 2062, 2063, 2064, 2065, 2067, 2070, 25, 2075, 2076, 2077, 2078, 2080, 2082, 2083, 2084, 2085, 39, 2088, 2101, 2104, 2108, 2112, 2113, 2114, 2115, 2116, 2118, 2122, 2126, 2130, 2134, 2136, 2137, 2140, 2141, 2142, 2144, 2147, 2148, 2151, 2152, 2154, 2162, 2167, 2171, 2174, 2175, 2176, 2178, 2179, 2180, 2185, 2187, 2192, 2194, 2202, 2204, 2207, 2208, 2209, 2210, 2215, 2216, 2217, 2218, 2219, 2224, 2226, 2233, 2234, 2236, 2237, 2238, 2241, 2244, 2250, 2251, 2257, 2268, 2273, 2274, 2282, 2284, 2286, 2287, 2288, 2296, 2299, 2303, 2304, 2308, 2310, 2314, 2315, 2321, 2322, 2323, 2328, 2333, 2335, 2338, 290, 2341, 2345, 2349, 303, 305, 307, 308, 2357, 2358, 309, 2362, 2363, 316, 2369, 2371, 324, 2376, 2377, 330, 2380, 2382, 2384, 336, 2386, 339, 2388, 340, 2391, 344, 2393, 2396, 2397, 2398, 2400, 2402, 2403, 358, 359, 2408, 2406, 2411, 2414, 2416, 2417, 2422, 2425, 2427, 2428, 2431, 2433, 2439, 2440, 2444, 2447, 2448, 2457, 569, 572, 573, 580, 586, 593, 594, 597, 599, 600, 620,

In [ ]:
def BM25_score(query, index, N, doc_lengths, avg_doc_length, k1 = 1.2, b = 0.75):
    bm25 = Counter()
    for term in query:

        # calculate idf
        try:
            df = index.df[term]
        except:
            continue
        idf = math.log(N/df, 10)

        # calculate bm25 score
        pl = index.read_a_posting_list('', term)
        for id, tf in pl:
            try:
                norm = (tf*(k1+1))/(tf+k1*(1-b+b*(doc_lengths[id]/avg_doc_length)))
                bm25[id] += idf*norm
            except:
                pass
    bm25_final = bm25.most_common()
    return bm25_final